XGBOOST (EXTREME GRADIENT BOOSTING)

GBM algoritmasının hız ve tahmin performansını arttırmak üzere optimize edilmiş; ölçeklenebilir ve farklı platformlarda entegre edilebilir halidir.

XGBoost, ağaç tabanlı bir öğrenme algoritmasıdır ve özellikle yapılandırılmış verilerle çalışmak için güçlüdür.

In [2]:
# "diabetes" verisetini yüklüyoruz.
import pandas as pd
import numpy as np
diabetes = pd.read_csv("diabetes.csv")
df = diabetes.copy()
df.head()

from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# Veriseti içindeki eksik verileri siliyoruz.
df = df.dropna()
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Verisetini tanımak için incelemeler yapıyoruz.

In [5]:
# Verisetindeki sayısal sütunlar için istatistiksel özet bilgilerini inceliyoruz.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [6]:
# Veriseti içinde eksik değer kalıp kalmadığını inceliyoruz.
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
# Verisetindeki bağımlı ve bağımsız değişkenleri belirliyoruz.
X = df.drop(["Outcome"], axis=1)
y = df["Outcome"]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [10]:
from xgboost import XGBClassifier

In [11]:
# Modeli oluşturduk ve eğittik.
model = XGBClassifier()
xgb_model = model.fit(X_train, y_train)

In [12]:
# Model üzerinde tahmin işlemini gerçekleştirdik.
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
# Test Seti için "accuracy_score" değerini hesapladık.
accuracy_score_test = accuracy_score(y_test, y_test_pred)
print('Test Seti Doğruluk Oranı:', accuracy_score_test)

Test Seti Doğruluk Oranı: 0.7272727272727273


Model Tuning

In [16]:
# Cross Validation (Çapraz Doğrulama) işlemini gerçekleştirmek için nesne oluşturuyoruz.
from sklearn.model_selection import GridSearchCV
xgb_params = {'n_estimators': [100, 500, 1000, 2000],
              'subsample': [0.6, 0.8, 1.0],
              'max_depth': [3, 4, 5,6],
              'learning_rate': [0.1,0.01,0.02,0.05],
              'min_samples_split': [2,5,10]}

In [18]:
# Cross Validation nesnesi oluşturduktan sonra modeli oluşturduk ve eğittik.
xgb_cv_model = GridSearchCV(xgb_model, xgb_params, cv = 10, n_jobs = -1, verbose = 2)
xgb_cv_model = xgb_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


In [19]:
# Model için en iyi parametreler aşağıdaki gibidir.
xgb_cv_model.best_params_

{'learning_rate': 0.05,
 'max_depth': 3,
 'min_samples_split': 2,
 'n_estimators': 100,
 'subsample': 1.0}

In [20]:
# Final Modelini kuruyoruz.
xgb_tuned = XGBClassifier(learning_rate = 0.05, 
                    max_depth = 3,
                    min_samples_split = 2,
                    n_estimators = 100,
                    subsample = 1)

xgb_tuned = xgb_tuned.fit(X_train,y_train)

In [21]:
# Final modeli üzerinde tahmin işlemini gerçekleştirdik.
y_pred_test = xgb_tuned.predict(X_test)

In [22]:
# Model tuning işleminden sonra "accuracy_score" değerini tekrar hesapladık.
accuracy_score_test = accuracy_score(y_test, y_pred_test)
print('Test Seti Doğruluk Oranı:', accuracy_score_test)

Test Seti Doğruluk Oranı: 0.7662337662337663
